# Import the necessary packages

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
import re

# Imread the csv information

In [ ]:
def read_csv_file(file_path):
    servoing_informations = []
    counter = 0
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # skip header

        for row in csv_reader:
            if counter == 0:
                goal_informations = row
            else:
                servoing_informations.append(row)
            counter += 1
    return goal_informations, servoing_informations

In [ ]:
goal_informations, servoing_informations= read_csv_file("/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/keypoints_servoing.csv")

# Get the time values

In [ ]:
interference_neural_networks = [] # only nn
time_for_keypoint_extraction = [] # keypoint extraction function
time_for_servoing_loop = [] # hole servoing loop
time_for_picture = [] # time for taking a picture

for i in range(len(servoing_informations)):
    time_for_servoing_loop.append(float(servoing_informations[i][2]))
    interference_neural_networks.append(float(servoing_informations[i][3]))
    time_for_keypoint_extraction.append(float(servoing_informations[i][4]))
    time_for_picture.append(float(servoing_informations[i][6]))


In [ ]:
# Generate random data for mean values (example)
mean_interference_neural_networks = np.array(interference_neural_networks).mean()
mean_time_for_keypoint_extraction = np.array(time_for_keypoint_extraction).mean()
mean_time_for_servoing_loop = np.array(time_for_servoing_loop).mean()
mean_time_for_picture = np.array(time_for_picture).mean()

print("Mean Interference Neural Network: ", mean_interference_neural_networks)
print("Mean Time for Interference + Keypoint Extraction: ", mean_time_for_keypoint_extraction)
print("Mean Time for Servoing Loop: ", mean_time_for_servoing_loop)
print("Mean Time for Taking a Picture: ", mean_time_for_picture)


# Create the figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

# Plot the upper code on the left subplot (ax1)
ax1.plot(interference_neural_networks, label="Interference Neural Network")
ax1.plot(time_for_keypoint_extraction, label="Time for Interference + Keypoint Extraction")
ax1.plot(time_for_servoing_loop, label="Time for Servoing Loop")
ax1.plot(time_for_picture, label="Time for Taking a Picture")
ax1.set_xlabel("Servoing Step", fontsize=14)
ax1.set_ylabel("Time in s", fontsize=14)
legend = ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2, fontsize=14)
ax1.set_ylim(0, max(mean_interference_neural_networks, mean_time_for_keypoint_extraction, mean_time_for_servoing_loop)*1.4)
ax1.title.set_text('Time Values while Servoing')
ax1.title.set_fontsize(18)
ax1.grid()

# Plot the mean values on the right subplot (ax2)
ax2.axhline(y=mean_interference_neural_networks, color="blue",label="Mean Interference Neural Network")
ax2.axhline(y=mean_time_for_keypoint_extraction, color="orange",label="Mean Time for Interference + Keypoint Extraction")
ax2.axhline(y=mean_time_for_servoing_loop, color="green",label="Mean for Servoing Loop")
ax2.axhline(y=mean_time_for_picture, color="red",label="Mean for Taking a Picture")
ax2.set_ylabel("Time in s", fontsize=14)
legend = ax2.legend(loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2, fontsize=14)
ax2.set_ylim(0, max(mean_interference_neural_networks, mean_time_for_keypoint_extraction, mean_time_for_servoing_loop)*1.4)
ax2.title.set_text('Mean Time Values')
ax2.title.set_fontsize(18)
ax2.grid()

# Display the plot
plt.suptitle('Time Values while Servoing', fontsize=24)
plt.show()

# Get the Progression of the Keypoints while Servoing

In [ ]:
goal_featurepoints = goal_informations[0]
goal_featurepoints = eval(goal_featurepoints)

In [ ]:
# make a list with 64 lists in it
pixel_error = []
for i in range(len(goal_featurepoints)):
    pixel_error.append([])

In [ ]:
for row in servoing_informations:
    featurepoints = row[0]
    featurepoints = eval(featurepoints)

    for i in range(len(featurepoints)):
        pixel_error_x = goal_featurepoints[i][0]-featurepoints[i][0]
        pixel_error_y = goal_featurepoints[i][1]-featurepoints[i][1]
        pixel_error[i].append([pixel_error_x, pixel_error_y])

In [ ]:
keypoint_number = 1 # starts with 0

plt.plot(pixel_error[keypoint_number])
plt.xlabel('Servoing Step')
plt.ylabel('Pixel Error')
plt.title('Pixel Error of Keypoint ' + str(keypoint_number))
plt.legend(['x-error of Keypoint '+str(keypoint_number), 'y-error of the Keypoint '+str(keypoint_number)], loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2)
plt.grid()
plt.show()



In [ ]:
num_keypoints = len(goal_featurepoints)

plt.figure(figsize=(10, 6))  # Optional: Set the size of the figure

for keypoint_number in range(num_keypoints):  # Assuming num_keypoints is the total number of keypoints
    plt.plot(pixel_error[keypoint_number])

plt.xlabel('Iterationes')
plt.ylabel('Pixel Error')
plt.title('Pixel Error of All Keypoints')

# Generate labels for the legend
legend_labels = ['Keypoint {}'.format(keypoint_number) for keypoint_number in range(num_keypoints)]

#plt.legend(legend_labels, loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2)
plt.grid()
plt.show()


# Plot the Velocity of the Endeffector while Servoing

In [ ]:
velocity = [] # hole servoing loop

for i in range(len(servoing_informations)):
    string = servoing_informations[i][5]
    clean_string = re.sub(r'[^\d.eE+-]+', ' ', string)
    values = re.findall(r"[-+]?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?", clean_string)
    float_values = [float(value) for value in values]
    velocity.append(float_values)

velocity = list(zip(*velocity))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 10))

ax1.plot(velocity[0])
ax1.set_title("Velocity in x-direction")
ax1.set_xlabel("Servoing Step", fontsize=16)
ax1.set_ylabel("Velocity in m/s", fontsize=16)


ax2.plot(velocity[1])
ax2.set_title("Velocity in y-direction")
ax2.set_xlabel("Servoing Step", fontsize=16)
ax2.set_ylabel("Velocity in m/s", fontsize=16)

plt.suptitle('Twist while Servoing', fontsize=24)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 2, figsize=(30, 20))
ax1[0].plot(velocity[0])
ax1[0].set_title("Velocity in x-direction")
ax1[0].set_xlabel("Servoing Step")
ax1[0].set_ylabel("Velocity in m/s")
ax1[0].legend()
ax1[0].grid()  

# Plot 2: Velocity in y-direction
ax1[1].plot(velocity[1])
ax1[1].set_title("Velocity in y-direction")
ax1[1].set_xlabel("Servoing Step")
ax1[1].set_ylabel("Velocity in m/s")
ax1[1].legend()
ax1[1].grid()

# Plot 3: Polynomial interpolation for Velocity in x-direction
x = np.arange(len(velocity[0]))
fit_coeffs_x = np.polyfit(x, velocity[0], deg=3)  # Adjust degree as needed
fit_x = np.polyval(fit_coeffs_x, x)
ax2[0].plot(x, velocity[0], label="Data")
ax2[0].plot(x, fit_x, label="Polynomial Fit")
ax2[0].set_title("Polynomial Interpolation for Velocity in x-direction")
ax2[0].set_xlabel("Servoing Step")
ax2[0].set_ylabel("Velocity in m/s")
ax2[0].legend()
ax2[0].grid()

# Plot 4: Polynomial interpolation for Velocity in y-direction
fit_coeffs_y = np.polyfit(x, velocity[1], deg=3)  # Adjust degree as needed
fit_y = np.polyval(fit_coeffs_y, x)
ax2[1].plot(x, velocity[1], label="Data")
ax2[1].plot(x, fit_y, label="Polynomial Fit")
ax2[1].set_title("Polynomial Interpolation for Velocity in y-direction")
ax2[1].set_xlabel("Servoing Step")
ax2[1].set_ylabel("Velocity in m/s")
ax2[1].legend()
ax2[1].grid()

plt.suptitle('Twist while Servoing', fontsize=24)
plt.tight_layout()
plt.show()



# Linarty of Key Points over time


In [ ]:
# We want to analyse the servoing informations[6]
lin_factor = []
for i in range(len(servoing_informations)):
    lin_factor.append(servoing_informations[i][7])

# Plot the linear factor
plt.figure(figsize=(15, 5))
plt.plot(lin_factor)
plt.title('Linear factor')
plt.xlabel('Iteration')
plt.ylabel('Linear factor')
plt.show()
    

# Zielbild und Enpose Visueller Vergleich

In [ ]:
zielbild = cv2.imread('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/goal_pic/goal_picture.png')
print(goal_featurepoints[1])

In [ ]:
import cv2
import numpy as np

# Das Bild einlesen
zielbild = cv2.imread('/home/scco0002/catkin_ws/src/iaps_ur5e_sim/scripts/kovis_moduls/img/goal_pic/goal_picture.png')

# Über jeden Punkt iterieren
for point in goal_featurepoints:
    # Den Punkt auf das Bild zeichnen
    zielbild = cv2.circle(zielbild, tuple(point), radius=0.2, color=(0,0,255), thickness=-1)

# Das bearbeitete Bild anzeigen
cv2.imshow('Zielbild', zielbild)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 